In [11]:
import gym
import numpy as np
import torch
from torch.nn import Module
from torch.nn import Linear
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.optim import Adam
import matplotlib.pyplot as plt

In [12]:
RANDOM_SEED = 99
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [13]:
env = gym.make('CartPole-v0')
env.seed(RANDOM_SEED)

[99]

In [14]:
# Свойста агента
STATES_SIZE =env.observation_space.shape[0]
ACTION_SIZE =env.action_space.n

In [17]:
class PolicyNet(Module):
    
    def __init__(self,input_size,output_size):
        super().__init__()
        
        self.fc1=Linear(input_size,32)
        self.fc2=Linear(32,output_size)
    
    def forward(self,state):
        
        x=self.fc1(x)
        x=F.relu(x)
        x=self.fc2(x)
        
        return F.softmax(x,dim=1)
    
    # Функция политики, возвращает действие на састояние и 
    # логорифм действия
    def act(self,state):
        props=self.forward(state) 
        prop_cat=Categorical(props)
        # Получаем действие
        action=prop_cat.sample()
        
        return action,prop_cat.log_prob(action)
        

In [ ]:
rewords=[]

In [18]:
train(net, optimizer):
    

SyntaxError: invalid syntax (1686138340.py, line 1)

In [19]:
policy = PolicyNet(STATES_SIZE,ACTION_SIZE)

In [21]:
optimizator = Adam(policy.parameters(),lr=0.01)

In [ ]:
for epoch in range(300):
    state = env.reset()
    action, log_prob = policy.act(state)
    state, reward, done, _ = env.step(action)
    rewords.append(reward)
    
    # Если игра завершинна
    if done:
        break
    